# L'ajout des noeuds dans la base de donnees neo4j

In [1]:
import json
import random
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "nadou24474979"))
collaborator_news = random.randint(20, 90)
manager_news = random.randint(20, 90)


def add_entire_structure_with_manager(tx, entreprise_name, direction_manager, service_manager, equipe_manager, direction_name, service_name, equipe_name, first_name, last_name, age, nb_news, email):
    # This Cypher query creates nodes for all entities and relationships between them
    query = """
    MERGE (entreprise:Entreprise {nom: $entreprise_name})
    MERGE (entreprise)-[:HAS_DIRECTION]->(direction:Structure {nom: $direction_name, type: 'direction'})
    MERGE (direction)-[:HAS_SERVICE]->(service:Structure {nom: $service_name, type: 'service'})
    MERGE (service)-[:HAS_EQUIPE]->(equipe:Structure {nom: $equipe_name, type: 'equipe'})
    MERGE (equipe)-[:HAS_MANAGER]->(equipe_manager:Manager {prenom: $equipe_manager_prenom, nom: $equipe_manager_nom, email: $equipe_manager_email, nb_news: $equipe_manager_nb_news})
    MERGE (direction)-[:HAS_MANAGER]->(direction_manager:Manager {prenom: $direction_manager_prenom, nom: $direction_manager_nom, email: $direction_manager_email, nb_news: $direction_manager_nb_news})
    MERGE (service)-[:HAS_MANAGER]->(service_manager:Manager {prenom: $service_manager_prenom, nom: $service_manager_nom, email: $service_manager_email, nb_news: $service_manager_nb_news})
    MERGE (employee:Collaborateur {prenom: $first_name, nom: $last_name, age: $age, email: $email, nb_news: $employee_nb_news})
    MERGE (equipe_manager)-[:MANAGES_EQUIPE]->(equipe)
    MERGE (direction_manager)-[:MANAGES_DIRECTION]->(direction)
    MERGE (service_manager)-[:MANAGES_SERVICE]->(service)
    MERGE (equipe)-[:HAS_EMPLOYEE]->(employee)
    """
    tx.run(query, entreprise_name=entreprise_name, direction_manager_prenom=direction_manager['prenom'], direction_manager_nom=direction_manager['nom'], direction_manager_email=direction_manager['email'], direction_manager_nb_news=direction_manager['nb_news'], service_manager_prenom=service_manager['prenom'], service_manager_nom=service_manager['nom'], service_manager_email=service_manager['email'], service_manager_nb_news=service_manager['nb_news'], equipe_manager_prenom=equipe_manager['prenom'], equipe_manager_nom=equipe_manager['nom'], equipe_manager_email=equipe_manager['email'], equipe_manager_nb_news=equipe_manager['nb_news'], direction_name=direction_name, service_name=service_name, equipe_name=equipe_name, first_name=first_name, last_name=last_name, age=age, email=email, employee_nb_news=nb_news)

def add_manager_par_relation(tx, direction_manager_nom, direction_manager_prenom, manager_par_nom, manager_par_prenom):
# This Cypher query creates a "MANAGER_PAR" relationship between two managers
    query = """
    MATCH (m1:Manager {nom: $direction_manager_nom, prenom: $direction_manager_prenom})
    MATCH (m2:Manager {nom: $manager_par_nom, prenom: $manager_par_prenom})
    MERGE (m1)-[:MANAGER_PAR]->(m2)
"""
    tx.run(query, direction_manager_nom=direction_manager_nom, direction_manager_prenom=direction_manager_prenom, manager_par_nom=manager_par_nom, manager_par_prenom=manager_par_prenom)

with open("./nom_prenom.json") as fichier_test1:
    data1 = json.load(fichier_test1)

with open('./Entreprise.json') as f:
    data = json.load(f)
for entreprise in data:
    entreprise_name = entreprise['nom']
    used_names = set()  # Set to keep track of used names
    used_emails = set()  # Set to keep track of used emails

    for direction in entreprise['children']:
        direction_name = direction['nom']
        
        # Randomly choose a manager for the direction
        aleatoire_direction_manager_prenom = random.choice(data1['prenom'])
        aleatoire_direction_manager_nom = random.choice(data1['nom'])
        aleatoire_direction_manager_email = aleatoire_direction_manager_prenom.lower() + '.' + aleatoire_direction_manager_nom.lower() + '@' + entreprise_name.lower().replace(' ', '') + '.com'

        direction_manager = {'prenom': aleatoire_direction_manager_prenom, 'nom': aleatoire_direction_manager_nom, 'email': aleatoire_direction_manager_email, 'nb_news': manager_news}

        # Add the direction manager to the used names and emails sets
        used_names.add((direction_manager['prenom'], direction_manager['nom']))
        used_emails.add(direction_manager['email'])

        for service in direction['children']:
            service_name = service['nom']

            # Randomly choose a manager for the service
            aleatoire_service_manager_prenom = random.choice(data1['prenom'])
            aleatoire_service_manager_nom = random.choice(data1['nom'])
            aleatoire_service_manager_email = aleatoire_service_manager_prenom.lower() + '.' + aleatoire_service_manager_nom.lower() + '@' + entreprise_name.lower().replace(' ', '') + '.com'

            service_manager = {'prenom': aleatoire_service_manager_prenom, 'nom': aleatoire_service_manager_nom, 'email': aleatoire_service_manager_email, 'nb_news': manager_news}

            # Add the service manager to the used names and emails sets
            used_names.add((service_manager['prenom'], service_manager['nom']))
            used_emails.add(service_manager['email'])

            for equipe in service['children']:
                equipe_name = equipe['nom']

                # Randomly choose a manager for the team
                aleatoire_equipe_manager_prenom = random.choice(data1['prenom'])
                aleatoire_equipe_manager_nom = random.choice(data1['nom'])
                aleatoire_equipe_manager_email = aleatoire_equipe_manager_prenom.lower() + '.' + aleatoire_equipe_manager_nom.lower() + '@' + entreprise_name.lower().replace(' ', '') + '.com'

                equipe_manager = {'prenom': aleatoire_equipe_manager_prenom, 'nom': aleatoire_equipe_manager_nom, 'email': aleatoire_equipe_manager_email, 'nb_news': manager_news}

                # Add the team manager to the used names and emails sets
                used_names.add((equipe_manager['prenom'], equipe_manager['nom']))
                used_emails.add(equipe_manager['email'])

                for i in range(5):
                    # Randomly choose a collaborator and generate a unique email
                    aleatoire_collaborateur_prenom = random.choice(data1['prenom'])
                    aleatoire_collaborateur_nom = random.choice(data1['nom'])
                    aleatoire_collaborateur_email = aleatoire_collaborateur_prenom.lower() + '.' + aleatoire_collaborateur_nom.lower() + '@' + entreprise_name.lower().replace(' ', '') + '.com'

                    # Check if the generated email is not already used
                    while aleatoire_collaborateur_email in used_emails:
                        aleatoire_collaborateur_prenom = random.choice(data1['prenom'])
                        aleatoire_collaborateur_nom = random.choice(data1['nom'])
                        aleatoire_collaborateur_email = aleatoire_collaborateur_prenom.lower() + '.' + aleatoire_collaborateur_nom.lower() + '@' + entreprise_name.lower().replace(' ', '') + '.com'

                    # Add the collaborator email to the used emails set
                    used_emails.add(aleatoire_collaborateur_email)

                    # Randomly generate the number of news for the collaborator
                    aleatoire_collaborateur_nb_news = random.randint(0, collaborator_news)

                    # Add the collaborator to the graph
                    with driver.session() as session:
                        session.execute_write(add_entire_structure_with_manager, entreprise_name, direction_manager, service_manager, equipe_manager, direction_name, service_name, equipe_name, aleatoire_collaborateur_prenom, aleatoire_collaborateur_nom, 30, aleatoire_collaborateur_nb_news, aleatoire_collaborateur_email)

                # Create "MANAGER_PAR" relationships between managers
                with driver.session() as session:
                    session.execute_write(add_manager_par_relation, direction_manager['nom'], direction_manager['prenom'], service_manager['nom'], service_manager['prenom'])
                    session.execute_write(add_manager_par_relation, service_manager['nom'], service_manager['prenom'], equipe_manager['nom'], equipe_manager['prenom'])

# Ajout de la date de début contrat et date de départ

In [2]:
from neo4j import GraphDatabase
import random
import json
from datetime import datetime, timedelta

# Connexion à la base de données Neo4j
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "nadou24474979"))

# Lecture du fichier JSON contenant les dates
with open("./Date_collaborateurs.json") as file:
    dates_data = json.load(file)

# Extraction des dates de début et de fin du contrat du fichier JSON
date_debut = datetime.strptime(dates_data["date_debut"], "%Y-%m-%d")
date_fin = datetime.strptime(dates_data["date_fin"], "%Y-%m-%d")

# Fonction pour générer une date aléatoire entre la date de début et la date de fin
def generate_random_date(start, end):
    delta = end - start
    random_days = random.randint(0, delta.days)
    random_date = start + timedelta(days=random_days)
    return random_date.strftime("%Y-%m-%d")

# Fonction pour ajouter les attributs à chaque collaborateur
def add_attributes_to_collaborateurs():
    with driver.session() as session:
        query = "MATCH (c:Collaborateur) RETURN c"
        result = session.run(query)

        # Parcourir chaque collaborateur
        for record in result:
            collaborateur = record["c"]

            # Vérifier si le collaborateur a une date de départ
            if "date_depart" in collaborateur:
                # Supprimer le collaborateur de la base de données
                delete_query = """
                MATCH (c:Collaborateur {nom: $nomCollab, prenom: $prenomCollab})
                DELETE c
                """
                session.run(delete_query, nomCollab=collaborateur["nom"], prenomCollab=collaborateur["prenom"])
                continue  # Passer au collaborateur suivant

            # Générer une date de début de contrat aléatoire
            start_date = generate_random_date(date_debut, date_fin)

            # Ajouter l'attribut "date_debut_contrat" au collaborateur
            update_query = """
            MATCH (c:Collaborateur {nom: $nomCollab, prenom: $prenomCollab})
            SET c.date_debut_contrat = $start_date
            """
            session.run(update_query, nomCollab=collaborateur["nom"], prenomCollab=collaborateur["prenom"], start_date=start_date)

            # Générer une probabilité de départ
            if random.random() <= 0.1:
                # Le collaborateur a quitté l'entreprise, générer une date de départ aléatoire
                departure_date = generate_random_date(date_debut, date_fin)

                # Ajouter l'attribut "date_depart" au collaborateur
                update_query = """
                MATCH (c:Collaborateur {nom: $nomCollab, prenom: $prenomCollab})
                SET c.date_depart = $departure_date
                """
                session.run(update_query, nomCollab=collaborateur["nom"], prenomCollab=collaborateur["prenom"], departure_date=departure_date)

# Appel de la fonction pour ajouter les attributs aux collaborateurs
add_attributes_to_collaborateurs()

# Fermeture de la connexion à la base de données Neo4j
driver.close()


# Le nombre de publications chaque heure dans un tableau

In [3]:
#
import json
from tabulate import tabulate
import random
import math
from datetime import datetime

# Ouvrir le fichier JSON des mois
with open("./Moyen_mois.json") as f:
    data = json.load(f)

# Ouvrir le fichier JSON des heures
with open("./heures.json") as f:
    data1 = json.load(f)
    
data1["Heures"]=list(range(24))
    
# Convertir les données en liste de listes pour le tabulateur
print("Tableau initial des mois avec leurs moyennes")
table_mois = []
for i in range(len(data["Mois"])):
    table_mois.append([data["Mois"][i], data["Moyen"][i]])

    
# Afficher le tableau avec tabulate
print(tabulate(table_mois, headers=["Mois", "Moyenne"]))
    
# print("Le tableau  ")
# T=[]
# moyenne_du_jour = []
# for i in range(len(data["Mois"])):
#    moyenne = data["Moyen"][i]
#    moyenne_du_jour = moyenne + random.randint(-2, 2)
#    mois_new=data["Mois"][i]
#    T.append([mois_new,moyenne_du_jour])
# Afficher le résultat
# print(tabulate(T, headers=["un jour du mois", "Moyenne"]))



table_initiale_heures = []

for i in range(len(data1["Heures"])):
    table_initiale_heures.append([data1["Heures"][i], data1["Pourcentage"][i]])
print(tabulate(table_initiale_heures, headers=["Heures", "Pourcentage"]))

# Créer une copie du tableau initial
table_copy_heures = table_initiale_heures.copy()

# Trier aléatoirement la copie du tableau
random.shuffle(table_copy_heures)

# Afficher la copie triée aléatoirement avec tabulate
print(tabulate(table_copy_heures, headers=["Heures", "Pourcentage"]))



def rand_round (num):
    a = random.randint(0, 1)
    if(a==1):
        return math.ceil(num)
    else :
        return math.floor(num)
    
def get_month_from_date(date):
    month = date.month
    return month

# le nb de publications chaque heure dans un tableau
def plublications_par_heure(date):
    
    #nb de publications par heure
    resultats = []
    
    i = get_month_from_date(date) - 1
    mois = table_mois[i][0]
    moyenne_mois = table_mois[i][1]
    #print("moyenne du mois", moyenne_mois)
    rand_variation = random.randint(-2, 2)
    #print("variation aléatoire", rand_variation)
    nb_pub = rand_round( moyenne_mois + rand_variation)
    print("nombre de publications pour cette journée: ",nb_pub)
    reste = nb_pub
    for j in range(len(table_copy_heures)):
        heure = table_copy_heures[j][0]
        pourcentage = table_copy_heures[j][1]
        resultat = rand_round((nb_pub * pourcentage / 100))  # Calculer le résultat arrondi à l'entier le plus proche
        if resultat <= reste :
            resultats.append([heure, resultat])
            reste = reste - resultat
        else :
            resultats.append([heure, reste])
            reste = 0
    if reste != 0:
        resultats[len(resultats)-1][1] += reste

    sortedresults = sorted(resultats, key=lambda x: x[0])
    
    finalresults = [x[1] for x in sortedresults]
    
    # Afficher le tableau final results
    print(finalresults)
    
    return finalresults


plublications_par_heure(datetime(2023, 6, 18))


Tableau initial des mois avec leurs moyennes
Mois         Moyenne
---------  ---------
Janvier        20
fevrier        19.28
mars           17.45
avril          18.1
mai            19.5
juin           13.5
juillet        14.8
aout           12.6
septembre      14
october        15
novembre       18
decembre       16
  Heures    Pourcentage
--------  -------------
       0            0.1
       1            0.1
       2            0.1
       3            0.1
       4            0.1
       5            0.1
       6            0.1
       7            0.1
       8            3.1
       9            9.9
      10           10.8
      11           13.6
      12           11.8
      13            7.6
      14            7
      15            6.8
      16            8.9
      17            9.2
      18            4.1
      19            2.8
      20            1.9
      21            0.9
      22            0.5
      23            0.3
  Heures    Pourcentage
--------  -------------
      18   

[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 3, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0]

# Ajout des publications 

In [4]:
import json
import datetime
import random
from neo4j import GraphDatabase
from datetime import datetime
from datetime import timedelta
import random
from datetime import time
from py2neo import Graph, Node, Relationship
from neo4j import GraphDatabase, basic_auth

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "nadou24474979"))

query = """MATCH (c:Collaborateur)
WITH c, RAND() AS r
ORDER BY r
LIMIT 1
RETURN c"""


def get_collab():
    with driver.session() as session:
        result = session.run(query)
        collab = result.single()["c"]
        return collab


# Charger les dates de début et de fin à partir du fichier JSON
with open("./date.json", "r") as file:
    dates_data = json.load(file)
date_debut_str = dates_data["date_debut"]
date_fin_str = dates_data["date_fin"]

# Convertir les dates de chaînes de caractères en objets datetime
date_debut = datetime.strptime(date_debut_str, "%Y-%m-%d")
date_fin = datetime.strptime(date_fin_str, "%Y-%m-%d")


def iterate_datetime(start_datetime, end_datetime):
    current_datetime = start_datetime
    onehour = timedelta(hours=1)

    while current_datetime <= end_datetime:
        pub_per_h = plublications_par_heure(current_datetime)
        for h in range(24):
            hour_range = f"{h:02d}:00-{h:02d}:59"
            current_hour = current_datetime.hour
            print(current_datetime.date(), hour_range, pub_per_h[current_hour])

            random_value = random.randint(1, 100)
            nb = pub_per_h[current_hour]
            for i in range(nb):
                random_minute = random.randint(0, 59)
                random_hour = current_hour
                random_time = current_datetime.replace(hour=random_hour, minute=random_minute)

                if pub_per_h[current_hour] > 0:
                    collaborateur = get_collab()
                    random_value = random.randint(1, 100)
                    while random_value >= collaborateur["nb_news"]:
                        collaborateur = get_collab()
                        random_value = random.randint(1, 100)
                    
                    date_debut_contrat = datetime.strptime(collaborateur["date_debut_contrat"], "%Y-%m-%d").date()
                    
                    if random_time.date() > date_debut_contrat:
                        print(f"{collaborateur['prenom']} {collaborateur['nom']} a créé une publication")
                        
                        with driver.session() as session:
                            query = '''
                            MATCH (c:Collaborateur {nom: $nomCollab, prenom: $prenomCollab}) 
                            CREATE (p:Publication {heure: $heure, date_creation: $date_creation})-[:A_CREE_PAR]->(c)
                            '''
                            session.run(query, nomCollab=collaborateur["nom"], prenomCollab=collaborateur["prenom"],
                                        heure=str(random_time), date_creation=str(current_datetime.date()))

            current_datetime += onehour

iterate_datetime(date_debut, date_fin)


nombre de publications pour cette journée:  20
[0, 0, 0, 1, 0, 0, 0, 1, 1, 2, 3, 2, 2, 2, 1, 1, 1, 2, 1, 0, 0, 0, 0, 0]
2023-01-01 00:00-00:59 0
2023-01-01 01:00-01:59 0
2023-01-01 02:00-02:59 0
2023-01-01 03:00-03:59 1
2023-01-01 04:00-04:59 0
2023-01-01 05:00-05:59 0
2023-01-01 06:00-06:59 0
2023-01-01 07:00-07:59 1
2023-01-01 08:00-08:59 1
2023-01-01 09:00-09:59 2
2023-01-01 10:00-10:59 3
2023-01-01 11:00-11:59 2
2023-01-01 12:00-12:59 2
2023-01-01 13:00-13:59 2
2023-01-01 14:00-14:59 1
2023-01-01 15:00-15:59 1
2023-01-01 16:00-16:59 1
2023-01-01 17:00-17:59 2
2023-01-01 18:00-18:59 1
2023-01-01 19:00-19:59 0
2023-01-01 20:00-20:59 0
2023-01-01 21:00-21:59 0
2023-01-01 22:00-22:59 0
2023-01-01 23:00-23:59 0
nombre de publications pour cette journée:  21
[0, 0, 0, 0, 0, 0, 0, 1, 1, 3, 3, 3, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0]
2023-01-02 00:00-00:59 0
2023-01-02 01:00-01:59 0
2023-01-02 02:00-02:59 0
2023-01-02 03:00-03:59 0
2023-01-02 04:00-04:59 0
2023-01-02 05:00-05:59 0
2023-01-02

# Ajouter point pour chaque collaborateur les points

In [5]:
from py2neo import Graph
import schedule
import time
import threading

# Connexion à la base de données Neo4j
graph = Graph("bolt://localhost:7687", auth=("neo4j", "nadou24474979"))

# Fonction qui réinitialise les points de chaque collaborateur à 100
def reset_points():
    
    query = """
    MATCH (c:Collaborateur)
    SET c.nombre_de_points = 100
    """

   
    graph.run(query)

# Programmation de l'exécution de la fonction reset_points toutes les semaines
schedule.every().monday.at("00:00").do(reset_points)

# Fonction pour gérer la planification dans un thread séparé
def run_schedule():
    while True:
        schedule.run_pending()
        time.sleep(1)

# Démarrage du thread de planification
schedule_thread = threading.Thread(target=run_schedule)
schedule_thread.start()




# Ajout les valeurs d'entreprise dans la base de données Neo4j

In [6]:
from neo4j import GraphDatabase
import random
import json

# Connexion à la base de données Neo4j
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "nadou24474979"))


def perform_action():
    with driver.session() as session:
        # Sélection aléatoire d'un collaborateur
        query = """
        MATCH (c:Collaborateur)
        WITH c, RAND() AS r
        ORDER BY r
        LIMIT 1
        RETURN c
        """
        result = session.run(query)
        collab = result.single()["c"]

        # Détermination de l'action
        action = random.choices(["gift", "news", "nothing"], weights=[0.3, 0.2, 0.5], k=1)[0]

        if action == "gift":
            # Sélection aléatoire d'un modèle de cadeau
            query = "MATCH (g:Gift_Model) RETURN g"
            result = session.run(query)
            gifts = [record["g"] for record in result]
            gift = random.choice(gifts)

            # Vérification si le collaborateur a suffisamment de points
            if collab["nombre_de_points"] >= gift["cost"]:
                # Sélection aléatoire d'une ou plusieurs valeurs
                query = "MATCH (v:Value) RETURN v"
                result = session.run(query)
                values = [record["v"] for record in result]
                chosen_values = random.choices(values, k=random.randint(1, len(values)))

                # Enregistrement du cadeau dans la base de données et déduction du coût
                query = f"""
                MATCH (c:Collaborateur {{name: '{collab['name']}', prenom: '{collab['prenom']}}})
                CREATE (g:Gift {{model: '{gift['name']}', cost: {gift['cost']}}})
                CREATE (c)-[:SENDS]->(g)
                SET c.nombre_de_points = c.nombre_de_points - {gift['cost']}
                """
                session.run(query)

                for value in chosen_values:
                    query = f"""
                    MATCH (g:Gift {{model: '{gift['name']}', cost: {gift['cost']}}})
                    MATCH (v:Value {{name: '{value['name']}}})
                    CREATE (g)-[:HAS_VALUE]->(v)
                    """
                    session.run(query)
perform_action()
        

# ----------------------------------------------------------------------

In [125]:
#l'ajout de publication par collaborateur 
import json
import datetime
import random
from neo4j import GraphDatabase
from datetime import datetime
from datetime import timedelta
import random
from datetime import time
from py2neo import Graph, Node, Relationship
from neo4j import GraphDatabase, basic_auth
uri = "bolt://localhost:7687"
graph = GraphDatabase.driver(uri, auth=("neo4j", "nadou24474979"))

#collaborateurs = []
query = """MATCH (c:Collaborateur)
WITH c, RAND() AS r
ORDER BY r
LIMIT 1
RETURN c"""



def get_collab():
     with driver.session() as session:
        result = session.run(query)
        collab = result.single()["c"]  # Utilisation de la méthode single() pour obtenir le collaborateur unique
        return collab


# Charger les dates de début et de fin à partir du fichier JSON
with open("/Users/ESPRI/Desktop/PFE/Neo4j/date.json", "r") as file:
    dates_data = json.load(file)
date_debut_str = dates_data["date_debut"]
date_fin_str = dates_data["date_fin"]

# Convertir les dates de chaînes de caractères en objets datetime
date_debut = datetime.strptime(date_debut_str, "%Y-%m-%d")
date_fin = datetime.strptime(date_fin_str, "%Y-%m-%d")



def iterate_datetime(start_datetime, end_datetime):
    current_datetime = start_datetime
    onehour = timedelta(hours=1)
    
    
    
    while current_datetime <= end_datetime:
        pub_per_h = plublications_par_heure(current_datetime)
        for h in range(24):
            hour_range = f"{h:02d}:00-{h:02d}:59"
            current_hour = current_datetime.hour
            print(current_datetime.date(), hour_range, pub_per_h[current_hour])

            random_value = random.randint(1, 100)
            nb = pub_per_h[current_hour]
            for i in range(nb):
                # Obtenir une heure aléatoire dans l'intervalle courant
                random_minute = random.randint(0, 59)
                random_hour = current_hour
                random_time = current_datetime.replace(hour=random_hour, minute=random_minute)
                if pub_per_h[current_hour] > 0:
                    collaborateur = get_collab()
                    random_value = random.randint(1, 100)
                    while random_value >= collaborateur["nb_news"]:
                        collaborateur = get_collab()
                        random_value = random.randint(1, 100)
                    print(f"{collaborateur['prenom']} {collaborateur['nom']} a créé une publication")
                    
                    # Ajouter la publication dans Neo4j avec une relation au collaborateur
                    with driver.session() as session:
                        query = '''
                        MATCH (c:Collaborateur {nom: $nomCollab, prenom: $prenomCollab}) 
                        CREATE (p:Publication {heure: $heure, date_creation: $date_creation})-[:A_CREE_PAR]->(c)
                        '''
                        session.run(query, nomCollab=collaborateur["nom"],prenomCollab=collaborateur["prenom"] ,heure=str(random_time), date_creation=str(current_datetime.date()))

            current_datetime += onehour

iterate_datetime(date_debut, date_fin)


nombre de publications pour cette journée:  20
[0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 2, 3, 1, 0, 1, 1, 2, 0, 0, 1, 1, 0, 1]
2023-01-01 00:00-00:59 0
2023-01-01 01:00-01:59 0
2023-01-01 02:00-02:59 1
Bouazize Ahmed a créé une publication
2023-01-01 03:00-03:59 0
2023-01-01 04:00-04:59 1
Saidi Roua a créé une publication
2023-01-01 05:00-05:59 1
Kharrat Amin a créé une publication
2023-01-01 06:00-06:59 0
2023-01-01 07:00-07:59 1
Dammak Mohamed a créé une publication
2023-01-01 08:00-08:59 1
Moalla Assewer a créé une publication
2023-01-01 09:00-09:59 1
Jabeur Hamdi a créé une publication
2023-01-01 10:00-10:59 1
Cherif Roua a créé une publication
2023-01-01 11:00-11:59 2
Mallouli Samy a créé une publication
Mallouli Roua a créé une publication
2023-01-01 12:00-12:59 3
Moalla Assewer a créé une publication
Mallouli Samy a créé une publication
Bouazize Ahmed a créé une publication
2023-01-01 13:00-13:59 1
Dammak Fatma a créé une publication
2023-01-01 14:00-14:59 0
2023-01-01 15:00-15:59 1
Be

2023-01-06 19:00-19:59 0
2023-01-06 20:00-20:59 1
Gouiaa Hanen a créé une publication
2023-01-06 21:00-21:59 1
Menif Fedi a créé une publication
2023-01-06 22:00-22:59 0
2023-01-06 23:00-23:59 1
Chouaibi Hichem a créé une publication
nombre de publications pour cette journée:  21
[1, 0, 1, 0, 0, 1, 0, 0, 0, 2, 0, 3, 3, 1, 0, 2, 2, 2, 0, 0, 1, 0, 1, 1]
2023-01-07 00:00-00:59 1
Abdelhedi Oumaima a créé une publication
2023-01-07 01:00-01:59 0
2023-01-07 02:00-02:59 1
Menif Yassmine a créé une publication
2023-01-07 03:00-03:59 0
2023-01-07 04:00-04:59 0
2023-01-07 05:00-05:59 1
Ben Hmida Dani a créé une publication
2023-01-07 06:00-06:59 0
2023-01-07 07:00-07:59 0
2023-01-07 08:00-08:59 0
2023-01-07 09:00-09:59 2
Chelly Hichem a créé une publication
Ben rejeb Heni a créé une publication
2023-01-07 10:00-10:59 0
2023-01-07 11:00-11:59 3
Ben amar Rahma a créé une publication
Hajkacem Karim a créé une publication
Wali Yosra a créé une publication
2023-01-07 12:00-12:59 3
Chelly Hanen a créé

Trabelsi Bacem a créé une publication
2023-01-12 16:00-16:59 1
Makni Hamdi a créé une publication
2023-01-12 17:00-17:59 1
Mallouli Roua a créé une publication
2023-01-12 18:00-18:59 0
2023-01-12 19:00-19:59 0
2023-01-12 20:00-20:59 0
2023-01-12 21:00-21:59 0
2023-01-12 22:00-22:59 0
2023-01-12 23:00-23:59 1
Makni Hamdi a créé une publication
nombre de publications pour cette journée:  20
[1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 2, 2, 3, 2, 0, 1, 1, 2, 1, 0, 1, 0, 0, 1]
2023-01-13 00:00-00:59 1
krichen Majdi a créé une publication
2023-01-13 01:00-01:59 0
2023-01-13 02:00-02:59 1
Wali Mohamed a créé une publication
2023-01-13 03:00-03:59 0
2023-01-13 04:00-04:59 0
2023-01-13 05:00-05:59 0
2023-01-13 06:00-06:59 1
Makni Hamdi a créé une publication
2023-01-13 07:00-07:59 0
2023-01-13 08:00-08:59 0
2023-01-13 09:00-09:59 1
Bouraoui Samy a créé une publication
2023-01-13 10:00-10:59 2
Wali Yosra a créé une publication
Dridi Oumaima a créé une publication
2023-01-13 11:00-11:59 2
Gouiaa Fatma a créé

2023-01-18 10:00-10:59 0
2023-01-18 11:00-11:59 2
Wali Yosra a créé une publication
Lahmer Yosr a créé une publication
2023-01-18 12:00-12:59 3
Samet Yassine a créé une publication
Kharrat Amin a créé une publication
Moalla Assewer a créé une publication
2023-01-18 13:00-13:59 1
Dridi Oumaima a créé une publication
2023-01-18 14:00-14:59 0
2023-01-18 15:00-15:59 1
Wali Amin a créé une publication
2023-01-18 16:00-16:59 1
Moalla Assewer a créé une publication
2023-01-18 17:00-17:59 2
krichen Majdi a créé une publication
Kharrat Amin a créé une publication
2023-01-18 18:00-18:59 1
Jabeur Hamdi a créé une publication
2023-01-18 19:00-19:59 0
2023-01-18 20:00-20:59 1
Chelly Hanen a créé une publication
2023-01-18 21:00-21:59 1
Lahmer Heni a créé une publication
2023-01-18 22:00-22:59 0
2023-01-18 23:00-23:59 1
Hajkacem Karim a créé une publication
nombre de publications pour cette journée:  21
[1, 0, 0, 0, 1, 0, 1, 0, 0, 3, 2, 3, 2, 1, 0, 2, 1, 2, 0, 1, 0, 0, 1, 0]
2023-01-19 00:00-00:59 1

krichen Majdi a créé une publication
2023-01-24 13:00-13:59 1
Bouazize Ahmed a créé une publication
2023-01-24 14:00-14:59 0
2023-01-24 15:00-15:59 1
Masmoudi Islem a créé une publication
2023-01-24 16:00-16:59 2
Saidi Yosra a créé une publication
Jallouli Karim a créé une publication
2023-01-24 17:00-17:59 1
Masmoudi Islem a créé une publication
2023-01-24 18:00-18:59 1
Jabeur Kholdoun a créé une publication
2023-01-24 19:00-19:59 0
2023-01-24 20:00-20:59 1
Ben Ameur Yosra a créé une publication
2023-01-24 21:00-21:59 1
Fakhfakh Islem a créé une publication
2023-01-24 22:00-22:59 1
Hajkacem Karim a créé une publication
2023-01-24 23:00-23:59 1
Menif Yosra a créé une publication
nombre de publications pour cette journée:  18
[0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 2, 3, 2, 1, 0, 1, 2, 1, 0, 0, 1, 1, 1, 0]
2023-01-25 00:00-00:59 0
2023-01-25 01:00-01:59 0
2023-01-25 02:00-02:59 0
2023-01-25 03:00-03:59 0
2023-01-25 04:00-04:59 0
2023-01-25 05:00-05:59 0
2023-01-25 06:00-06:59 1
Chouaibi Imen a c

2023-01-30 12:00-12:59 2
Jallouli Karim a créé une publication
Gouiaa Hanen a créé une publication
2023-01-30 13:00-13:59 2
krichen Fedi a créé une publication
Wali Amin a créé une publication
2023-01-30 14:00-14:59 0
2023-01-30 15:00-15:59 2
Ben Ayed Hamdi a créé une publication
Menif Fedi a créé une publication
2023-01-30 16:00-16:59 2
Abdelhedi Oumaima a créé une publication
Masmoudi Imen a créé une publication
2023-01-30 17:00-17:59 2
Fakhfakh Fatma a créé une publication
Ben Ameur Yosra a créé une publication
2023-01-30 18:00-18:59 1
krichen Nadine a créé une publication
2023-01-30 19:00-19:59 0
2023-01-30 20:00-20:59 1
Wali Amin a créé une publication
2023-01-30 21:00-21:59 0
2023-01-30 22:00-22:59 0
2023-01-30 23:00-23:59 0
nombre de publications pour cette journée:  21
[0, 0, 0, 0, 1, 1, 1, 1, 0, 3, 2, 3, 3, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0]
2023-01-31 00:00-00:59 0
2023-01-31 01:00-01:59 0
2023-01-31 02:00-02:59 0
2023-01-31 03:00-03:59 0
2023-01-31 04:00-04:59 1
Wali Yosra a c

Chelly Hanen a créé une publication
Bouazize Ahmed a créé une publication
2023-02-05 12:00-12:59 2
Chelly Hichem a créé une publication
Mallouli Roua a créé une publication
2023-02-05 13:00-13:59 1
Masmoudi Islem a créé une publication
2023-02-05 14:00-14:59 0
2023-02-05 15:00-15:59 1
Dridi Assad a créé une publication
2023-02-05 16:00-16:59 2
Tkaya Lina a créé une publication
Ben amar Oumaima a créé une publication
2023-02-05 17:00-17:59 1
Menif Yosra a créé une publication
2023-02-05 18:00-18:59 1
Jallouli Karim a créé une publication
2023-02-05 19:00-19:59 0
2023-02-05 20:00-20:59 0
2023-02-05 21:00-21:59 1
Tkaya Lina a créé une publication
2023-02-05 22:00-22:59 0
2023-02-05 23:00-23:59 1
Fakhfakh Yosra a créé une publication
nombre de publications pour cette journée:  20
[1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 3, 2, 2, 1, 0, 2, 2, 1, 1, 0, 0, 0, 1, 0]
2023-02-06 00:00-00:59 1
Fakhfakh Islem a créé une publication
2023-02-06 01:00-01:59 0
2023-02-06 02:00-02:59 1
krichen Karim a créé une pu

krichen Imen a créé une publication
2023-02-11 13:00-13:59 1
Trabelsi Bacem a créé une publication
2023-02-11 14:00-14:59 0
2023-02-11 15:00-15:59 1
Lahmer Yosr a créé une publication
2023-02-11 16:00-16:59 2
Hajkacem Karim a créé une publication
Trabelsi Assad a créé une publication
2023-02-11 17:00-17:59 1
Tkaya Lina a créé une publication
2023-02-11 18:00-18:59 1
Masmoudi Imen a créé une publication
2023-02-11 19:00-19:59 1
Ben amar Oumaima a créé une publication
2023-02-11 20:00-20:59 0
2023-02-11 21:00-21:59 0
2023-02-11 22:00-22:59 0
2023-02-11 23:00-23:59 1
Lahmer Islem a créé une publication
nombre de publications pour cette journée:  17
[1, 0, 0, 0, 0, 1, 1, 0, 0, 2, 0, 2, 2, 1, 0, 1, 2, 2, 1, 0, 1, 0, 0, 0]
2023-02-12 00:00-00:59 1
Menif Fedi a créé une publication
2023-02-12 01:00-01:59 0
2023-02-12 02:00-02:59 0
2023-02-12 03:00-03:59 0
2023-02-12 04:00-04:59 0
2023-02-12 05:00-05:59 1
Smaoui Amin a créé une publication
2023-02-12 06:00-06:59 1
Hajkacem Roua a créé une publ

Samet Nadine a créé une publication
Saidi Bilel a créé une publication
Ben rejeb Samy a créé une publication
2023-02-17 13:00-13:59 1
Ben amar Rahma a créé une publication
2023-02-17 14:00-14:59 2
chebbi Omar a créé une publication
Chelly Hichem a créé une publication
2023-02-17 15:00-15:59 1
Cherif Roua a créé une publication
2023-02-17 16:00-16:59 1
Fakhfakh Islem a créé une publication
2023-02-17 17:00-17:59 1
Moalla Assewer a créé une publication
2023-02-17 18:00-18:59 0
2023-02-17 19:00-19:59 1
Wali Amin a créé une publication
2023-02-17 20:00-20:59 1
Hajkacem Fatma a créé une publication
2023-02-17 21:00-21:59 1
Mallouli Samy a créé une publication
2023-02-17 22:00-22:59 0
2023-02-17 23:00-23:59 0
nombre de publications pour cette journée:  20
[0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 3, 3, 3, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]
2023-02-18 00:00-00:59 0
2023-02-18 01:00-01:59 0
2023-02-18 02:00-02:59 0
2023-02-18 03:00-03:59 0
2023-02-18 04:00-04:59 1
Jabeur Kholdoun a créé une publication
202

Dammak Mohamed a créé une publication
2023-02-23 08:00-08:59 0
2023-02-23 09:00-09:59 2
Ben rejeb Samy a créé une publication
Wali Amin a créé une publication
2023-02-23 10:00-10:59 0
2023-02-23 11:00-11:59 2
Dammak Fatma a créé une publication
Kharrat Omar a créé une publication
2023-02-23 12:00-12:59 3
Abdelhedi Oumaima a créé une publication
Mallouli Samy a créé une publication
Bouazize Ahmed a créé une publication
2023-02-23 13:00-13:59 2
Fakhfakh Islem a créé une publication
Menif Yassmine a créé une publication
2023-02-23 14:00-14:59 0
2023-02-23 15:00-15:59 2
Dridi Oumaima a créé une publication
Lahmer Islem a créé une publication
2023-02-23 16:00-16:59 0
2023-02-23 17:00-17:59 2
Jallouli Majdi a créé une publication
Ben Hmida Fatma a créé une publication
2023-02-23 18:00-18:59 1
Ben Ameur Yosra a créé une publication
2023-02-23 19:00-19:59 0
2023-02-23 20:00-20:59 0
2023-02-23 21:00-21:59 0
2023-02-23 22:00-22:59 0
2023-02-23 23:00-23:59 1
Mallouli Roua a créé une publication
n

In [ ]:
from neo4j import GraphDatabase
import random
import json
from datetime import datetime, timedelta

# Connexion à la base de données Neo4j
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "nadou24474979"))

# Lecture du fichier JSON contenant les dates
with open("./Date_collaborateurs.json") as file:
    dates_data = json.load(file)

# Extraction des dates de début et de fin du contrat du fichier JSON
date_debut = datetime.strptime(dates_data["date_debut"], "%Y-%m-%d")
date_fin = datetime.strptime(dates_data["date_fin"], "%Y-%m-%d")

# Fonction pour générer une date aléatoire entre la date de début et la date de fin
def generate_random_date(start, end):
    delta = end - start
    random_days = random.randint(0, delta.days)
    random_date = start + timedelta(days=random_days)
    return random_date.strftime("%Y-%m-%d")

# Fonction pour ajouter les attributs à chaque collaborateur
def add_attributes_to_collaborateurs():
    with driver.session() as session:
        query = "MATCH (c:Collaborateur) RETURN c"
        result = session.run(query)

        # Parcourir chaque collaborateur
        for record in result:
            collaborateur = record["c"]

            # Vérifier si le collaborateur a une date de départ
            if "date_depart" in collaborateur:
                # Supprimer le collaborateur de la base de données
                delete_query = """
                MATCH (c:Collaborateur {nom: $nomCollab, prenom: $prenomCollab})
                DELETE c
                """
                session.run(delete_query, nomCollab=collaborateur["nom"], prenomCollab=collaborateur["prenom"])
                continue  # Passer au collaborateur suivant

            # Générer une date de début de contrat aléatoire
            start_date = generate_random_date(date_debut, date_fin)

            # Ajouter l'attribut "date_debut_contrat" au collaborateur
            update_query = """
            MATCH (c:Collaborateur {nom: $nomCollab, prenom: $prenomCollab})
            SET c.date_debut_contrat = $start_date
            """
            session.run(update_query, nomCollab=collaborateur["nom"], prenomCollab=collaborateur["prenom"], start_date=start_date)

            # Générer une probabilité de départ
            if random.random() <= 0.1:
                # Le collaborateur a quitté l'entreprise, générer une date de départ aléatoire
                departure_date = generate_random_date(date_debut, date_fin)

                # Ajouter l'attribut "date_depart" au collaborateur
                update_query = """
                MATCH (c:Collaborateur {nom: $nomCollab, prenom: $prenomCollab})
                SET c.date_depart = $departure_date
                """
                session.run(update_query, nomCollab=collaborateur["nom"], prenomCollab=collaborateur["prenom"], departure_date=departure_date)

# Appel de la fonction pour ajouter les attributs aux collaborateurs
add_attributes_to_collaborateurs()

# Fermeture de la connexion à la base de données Neo4j
driver.close()


In [143]:
#Ajouter point pour chaque collaborateur 


from py2neo import Graph

# Connexion à la base de données Neo4j
graph = Graph("bolt://localhost:7687", auth=("neo4j", "nadou24474979"))

# Requête pour ajouter l'attribut "nombre de points" avec la valeur 100 à chaque collaborateur
query = """
MATCH (c:Collaborateur)
SET c.nombre_de_points = 100
"""

# Exécution de la requête
graph.run(query)


(No data)

In [ ]:
--------------------------------------------------------------------------------


In [171]:
import json
from datetime import datetime, timedelta
import random
from py2neo import Graph, Node, Relationship
from neo4j import GraphDatabase, basic_auth

uri = "bolt://localhost:7687"
graph = GraphDatabase.driver(uri, auth=("neo4j", "nadou24474979"))

# Initialize dictionary with hour ranges
publications_per_hour = {f"{str(i).zfill(2)}:00-{str(i).zfill(2)}:59": 0 for i in range(24)}

# Query to fetch publications
query = """MATCH (p:Publication)
WHERE p.date_creation = $date_creation
RETURN p"""

date_string = "2023-01-01"  # Replace this with the date you want

with graph.session() as session:
    result = session.run(query, date_creation=date_string)

    for record in result:
        publication = record["p"]

        # Extract hour from publication time
        hour = datetime.strptime(publication["heure"], "%Y-%m-%d %H:%M:%S").hour

        # Increment corresponding hour range in dictionary
        publications_per_hour[f"{str(hour).zfill(2)}:00-{str(hour).zfill(2)}:59"] += 1

# Print the dictionary in a format that suits you
for hour_range, count in publications_per_hour.items():
    print(f"{hour_range}: {count}")

00:00-00:59: 0
01:00-01:59: 0
02:00-02:59: 0
03:00-03:59: 0
04:00-04:59: 0
05:00-05:59: 0
06:00-06:59: 0
07:00-07:59: 0
08:00-08:59: 0
09:00-09:59: 0
10:00-10:59: 0
11:00-11:59: 0
12:00-12:59: 0
13:00-13:59: 0
14:00-14:59: 0
15:00-15:59: 0
16:00-16:59: 0
17:00-17:59: 0
18:00-18:59: 0
19:00-19:59: 0
20:00-20:59: 0
21:00-21:59: 0
22:00-22:59: 0
23:00-23:59: 0


In [71]:
#l'ajout de la tendance et l'amplitude a chaque collaborateur 
from py2neo import Graph

# Connexion à la base de données Neo4j
graph = Graph("bolt://localhost:7687", auth=("neo4j", "nadou24474979"))

# Récupérer tous les collaborateurs de la base de données
query = """
MATCH (c:Collaborateur)
RETURN c
"""
result = graph.run(query)

# Parcourir chaque collaborateur et effectuer les mises à jour
for record in result:
    collaborateur = record['c']

    # Générer la tendance aléatoire entre 30 et 100
    tendance = random.randint(30, 100)

    # Calculer l'amplitude en fonction de la tendance
    if random.random() * 100 < tendance:
        amplitude = random.uniform(0, 0.05)  # Entre 0% et 5%
    else:
        amplitude = random.uniform(-0.05, 0)  # Entre -5% et 0%

    # Calculer la motivation courante en utilisant l'amplitude et nb_news
    motivation_courante = collaborateur['nb_news'] * (1 + amplitude)

    # Mettre à jour les attributs dans la base de données
    collaborateur['tendance'] = tendance
    collaborateur['amplitude'] = amplitude
    collaborateur['motivationCourante'] = motivation_courante

    # Enregistrer les modifications dans la base de données
    graph.push(collaborateur)


In [45]:
import json
import random

# Lecture du fichier JSON de configuration
with open("chemin/vers/votre/config.json", "r") as file:
    config = json.load(file)
pourcentage = config["pourcentage"] / 100  # Convertir en décimal

# Lecture du fichier JSON des collaborateurs
with open("chemin/vers/votre/fichier.json", "r") as file:
    data = json.load(file)

# Choix aléatoire d'un collaborateur
collaborateur = random.choice(data["collaborateurs"])

# Récupération des attributs de tendance et d'amplitude
tendance_actuelle = collaborateur["tendance"]
amplitude_actuelle = collaborateur["amplitude"]

# Calcul des nouvelles valeurs de tendance et d'amplitude avec le pourcentage
nouvelle_tendance = tendance_actuelle + (tendance_actuelle * pourcentage)
nouvelle_amplitude = amplitude_actuelle + (amplitude_actuelle * pourcentage)

# Mise à jour des attributs de tendance et d'amplitude
collaborateur["tendance"] = nouvelle_tendance
collaborateur["amplitude"] = nouvelle_amplitude

# Écriture des nouvelles données dans le fichier JSON
with open("chemin/vers/votre/fichier.json", "w") as file:
    json.dump(data, file, indent=4)

# Message d'assurance
print(f"Les attributs de tendance et d'amplitude du collaborateur {collaborateur['nom']} ont été mis à jour avec une augmentation de {config['pourcentage']}% et enregistrés dans le fichier JSON.")


FileNotFoundError: [Errno 2] No such file or directory: 'chemin/vers/votre/config.json'

In [ ]:
#update de la tendance et l'amplitude 
from neo4j import GraphDatabase


# Requête pour récupérer un collaborateur au hasard avec ses attributs de tendance et d'amplitude
query = """
    MATCH (c:Collaborateur)
    WITH c, RAND() AS rand
    RETURN c, rand
    ORDER BY rand
    LIMIT 1
"""
with driver.session() as session:
    result = session.run(query)
    record = result.single()
    collaborateur = record["c"]
    tendance_actuelle = collaborateur["tendance"]
    amplitude_actuelle = collaborateur["amplitude"]

    # Calcul des nouvelles valeurs de tendance et d'amplitude
    nouvelle_tendance = tendance_actuelle + (tendance_actuelle * 0.02)  # 2%
    nouvelle_amplitude = amplitude_actuelle + (amplitude_actuelle * 0.02)  # 2%

    # Requête de mise à jour pour mettre à jour les attributs de tendance et d'amplitude
    update_query = """
        MATCH (c:Collaborateur)
        WHERE ID(c) = $collaborateur_id
        SET c.tendance = $nouvelle_tendance, c.amplitude = $nouvelle_amplitude
    """
    session.run(update_query, collaborateur_id=collaborateur["element_id"], nouvelle_tendance=nouvelle_tendance, nouvelle_amplitude=nouvelle_amplitude)


In [ ]:
import json
from neo4j import GraphDatabase

# Lecture du fichier JSON de configuration
with open("./pourcentage_amplitude_tendance.json", "r") as file:
    config = json.load(file)
pourcentage = config["pourcentage"] / 100  # Convertir en décimal



# Requête pour récupérer un collaborateur au hasard avec ses attributs de tendance et d'amplitude
query = """
    MATCH (c:Collaborateur)
    WITH c, RAND() AS rand
    RETURN c, rand
    ORDER BY rand
    LIMIT 1
"""
with driver.session() as session:
    result = session.run(query)
    record = result.single()
    collaborateur = record["c"]
    tendance_actuelle = collaborateur["tendance"]
    amplitude_actuelle = collaborateur["amplitude"]

    # Calcul des nouvelles valeurs de tendance et d'amplitude avec le pourcentage
    nouvelle_tendance = tendance_actuelle + (tendance_actuelle * pourcentage)
    nouvelle_amplitude = amplitude_actuelle + (amplitude_actuelle * pourcentage)

    # Requête de mise à jour pour mettre à jour les attributs de tendance et d'amplitude
    update_query = """
        MATCH (c:Collaborateur)
        WHERE ID(c) = $collaborateur_id
        SET c.tendance = $nouvelle_tendance, c.amplitude = $nouvelle_amplitude
    """
    session.run(update_query, collaborateur_id=collaborateur.id, nouvelle_tendance=nouvelle_tendance, nouvelle_amplitude=nouvelle_amplitude)

    # 
    print(f"Les attributs de tendance et d'amplitude du collaborateur {collaborateur.id} ont été mis à jour avec une augmentation de {config['pourcentage']}%.")


In [ ]:
from neo4j import GraphDatabase
import random
import json

# Connexion à la base de données Neo4j
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "nadou24474979"))

# Lecture du fichier JSON contenant les dates
with open("./Date_collaborateurs.json") as file:
    dates_data = json.load(file)

# Extraction des dates de début et de fin du contrat du fichier JSON
start_date = dates_data["date_debut"]
end_date = dates_data["date_fin"]

# Fonction pour générer une date aléatoire entre la date de début et la date de fin
def generate_random_date():
    start_timestamp = pd.Timestamp(start_date)
    end_timestamp = pd.Timestamp(end_date)
    random_timestamp = pd.Timestamp(
        random.randint(start_timestamp.value, end_timestamp.value)
    )
    random_date = random_timestamp.strftime("%Y-%m-%d")
    return random_date

# Fonction pour ajouter les attributs à chaque collaborateur
def add_attributes_to_collaborators():
    with driver.session() as session:
        # Récupérer tous les collaborateurs
        query = "MATCH (c:Collaborator) RETURN c"
        result = session.run(query)

        # Parcourir chaque collaborateur
        for record in result:
            collaborator = record["c"]

            # Générer une date de début de contrat aléatoire
            start_date = generate_random_date()

            # Ajouter l'attribut "date_debut_contrat" au collaborateur
            query = "MATCH (c:Collaborator {id: $id}) SET c.date_debut_contrat = $start_date"
            session.run(query, id=collaborator["id"], start_date=start_date)

            # Générer une probabilité de départ
            if random.random() <= 0.1:
                # Le collaborateur a quitté l'entreprise, générer une date de départ aléatoire
                departure_date = generate_random_date()

                # Ajouter l'attribut "date_depart" au collaborateur
                query = "MATCH (c:Collaborator {id: $id}) SET c.date_depart = $departure_date"
                session.run(query, id=collaborator["id"], departure_date=departure_date)

# Appel de la fonction pour ajouter les attributs aux collaborateurs
add_attributes_to_collaborators()

# Fermeture de la connexion à la base de données Neo4j
driver.close()


In [121]:
from neo4j import GraphDatabase
import random
import json
from datetime import datetime, timedelta

# Connexion à la base de données Neo4j
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "nadou24474979"))

# Lecture du fichier JSON contenant les dates
with open("./Date_collaborateurs.json") as file:
    dates_data = json.load(file)

# Extraction des dates de début et de fin du contrat du fichier JSON
date_debut = datetime.strptime(dates_data["date_debut"], "%Y-%m-%d")
date_fin = datetime.strptime(dates_data["date_fin"], "%Y-%m-%d")

# Fonction pour générer une date aléatoire entre la date de début et la date de fin
def generate_random_date(start, end):
    delta = end - start
    random_days = random.randint(0, delta.days)
    random_date = start + timedelta(days=random_days)
    return random_date.strftime("%Y-%m-%d")

# Fonction pour ajouter les attributs à chaque collaborateur
def add_attributes_to_collaborateurs():
    with driver.session() as session:
        query = "MATCH (c:Collaborateur) RETURN c"
        result = session.run(query)

        # Parcourir chaque collaborateur
        for record in result:
            collaborateur = record["c"]

            # Générer une date de début de contrat aléatoire
            start_date = generate_random_date(date_debut, date_fin)

            # Ajouter l'attribut "date_debut_contrat" au collaborateur
            query = """
            MATCH (c:Collaborateur {nom: $nomCollab, prenom: $prenomCollab})
            SET c.date_debut_contrat = $start_date
            """
            session.run(query, nomCollab=collaborateur["nom"], prenomCollab=collaborateur["prenom"], start_date=start_date)

            # Générer une probabilité de départ
            if random.random() <= 0.1:
                # Le collaborateur a quitté l'entreprise, générer une date de départ aléatoire
                departure_date = generate_random_date(date_debut, date_fin)

                # Ajouter l'attribut "date_depart" au collaborateur
                query = """
                MATCH (c:Collaborateur {nom: $nomCollab, prenom: $prenomCollab})
                SET c.date_depart = $departure_date
                """
                session.run(query, nomCollab=collaborateur["nom"], prenomCollab=collaborateur["prenom"], departure_date=departure_date)

# Appel de la fonction pour ajouter les attributs aux collaborateurs
add_attributes_to_collaborateurs()

# Fermeture de la connexion à la base de données Neo4j
driver.close()
